In [100]:
import pandas as pd
import requests
import json
import re

In [101]:
# Men, Women
cat = "Women"

"https://scores.bcci.tv/feeds-international/internationalarchives.js"

In [102]:
BCCI = {"BCCI_RECENT_MATCH_URL": f"https://scores2.bcci.tv/getRecentMatches?platform=international&previousMatchesCount=999&filterType={cat}&loadMore=true&archieves=true",
"BCCI_LIVE_MATCH_URL": f"https://scores2.bcci.tv/getLiveMatches?platform=international&previousMatchesCount=999&filterType={cat}&loadMore=true&archieves=true",
"BCCI_UPCOMING_MATCH_URL": f"https://scores2.bcci.tv/getUpcomingMatches?platform=international&previousMatchesCount=999&filterType={cat}&loadMore=true&archieves=true"}

CRICKET_AUS_COMMS = "https://apiv2.cricket.com.au/web/views/comments?fixtureId=20755&inningNumber=2&commentType=&overLimit=21&jsconfig=eccn%3Atrue&format=json"
# CRICKET_AUS_MATCH = "https://apiv2.cricket.com.au/web/fixtures/yearfilter?isCompleted=true&year=2025&limit=13&isInningInclude=true&jsconfig=eccn%3Atrue&format=json"

bcci_match_list = []
for i in BCCI.values():
    response = requests.get(i, timeout=100)
    data = response.json()
    rows = list(data.keys())
    if(rows[0] == 'recentMatches'):
        for j in data[list(data.keys())[-1]]:
            bcci_match_list.extend(j)
    else:
        bcci_match_list.extend(data[list(data.keys())[0]])

bcci_match_list = sorted(bcci_match_list, key = lambda x: x["MatchDate"])

with open(f"./bcci_shot_data/{cat}/bcci_match_list.json", 'w') as f:
    json.dump(bcci_match_list, f, indent=4)

In [103]:

def get_bcci_shot_data(match_id, max_overs):
    # match_id = 1653
    innings = 0

    if max_overs == 20 or max_overs == 50:
        innings = 2
    else:
        innings = 4

    match_data = []
    for i in range(1, innings+1):
        BCCI_COMMS_URL = f"https://scores.bcci.tv/feeds-international/scoringfeeds/{match_id}-Innings{i}.js"
        
        try:
            response = requests.get(BCCI_COMMS_URL, timeout=100)
            response.raise_for_status()
            data = response.text
            # result = re.sub(r'onScoring\((.*?)\);', r'\1', data)
            result = data.replace("onScoring(", "").replace(");", "")
            data_json = json.loads(result)
            match_data.append(data_json[f"Innings{i}"])

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 404:
                continue

    with open(f"./bcci_shot_data/{cat}/json/{match_id}.json", 'w') as f:
        json.dump(match_data, f, indent=4)

In [104]:
def csv_to_json(match_id):
    df = pd.read_json(f"./bcci_shot_data/{cat}/json/{match_id}.json")

    dfs = []
    for i in range(len(df['OverHistory'])):
        dfs.append(pd.DataFrame(df['OverHistory'][i]))
        
    final_df = pd.concat(dfs, axis=0, ignore_index=True)

    # final_df.to_csv(f"./bcci_shot_data/{match_id}.csv")
    final_df = final_df.drop(columns=['BallID', 'BallUniqueID', 'StrikerID', 'NonStrikerID', 'BowlerID', 
                                    'VideoFile', 'NewCommentry', 'Commentry', 'UPDCommentry', 'OutBatsManID',
                                    'HatCheck', 'CommentStrikers', 'OverName', 'CommentOver', 'RunsText'])


    final_df = final_df[final_df['ActualBallNo'].str.strip() != '']
    final_df.to_csv(f"./bcci_shot_data/{cat}/csv/{match_id}.csv")
    final_df.columns.unique()

In [105]:
temp_df = pd.DataFrame(bcci_match_list)
temp_df = temp_df.drop_duplicates(subset=['MatchID'], keep='first', inplace=False)

data_temp_df = temp_df[temp_df['MatchStatus'] == 'Post']

# for match_id ,max_overs in zip(data_temp_df['MatchID'], data_temp_df['MATCH_NO_OF_OVERS']):
#     get_bcci_shot_data(match_id, max_overs)
#     csv_to_json(match_id)

In [106]:
temp_df.fillna('', inplace=True)
temp_df = temp_df.drop(columns=['PreMatchCommentary', 'PostMatchCommentary'])
temp_df.to_csv(f'./bcci_shot_data/{cat}/bcci_match_list.csv', index=False)

/tmp/ipykernel_43660/3674046442.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  temp_df.fillna('', inplace=True)


In [107]:
import os
import pandas as pd

# Path to the folder containing the CSV files
folder_path = f'./bcci_shot_data/{cat}/csv'  # Path to the 'csv' directory

# List of numbers (in the order you want the CSV files to be combined)
temp_list = temp_df['MatchID'].tolist()

# List of all CSV files in the folder
csv_files = list(f"{num}.csv" for num in temp_list)  # Set for fast lookup

# Read and concatenate the CSVs
df_list = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    if os.path.exists(file_path):  # Only process if the file exists
        df = pd.read_csv(file_path)
        df_list.append(df)

# Concatenate all DataFrames into one
if df_list:
    final_df = pd.concat(df_list, ignore_index=True)
    # Save the result to a new CSV
    final_df.to_csv(f'./bcci_shot_data/{cat}/combined_shot_data.csv', index=False)
    print('CSV files have been concatenated successfully!')
else:
    print('No CSV files were found to combine.')


CSV files have been concatenated successfully!


In [108]:
def preprocess(data):
    df = pd.json_normalize(data['inning'], 'overs', ['id', 'fixtureId', 'inningNumber', 'battingTeamId', 'bowlingTeamId'], record_prefix='over_')
    # df1 = pd.json_normalize(df['balls'])
    # df = df.iloc[::-1].reset_index(drop=True)
    if(len(df) > 0):
        df = df.drop(index=df.index[-1])
        df = df.explode('over_balls', ignore_index=True)
        df1 = pd.json_normalize(df['over_balls'])
        df.drop(columns=['over_balls'], inplace=True)
        df2 = pd.concat([df, df1], axis=1)
        # df2['ball_comments'] = df2['comments'].apply(lambda x: ', '.join([comment['comments'] for comment in x]) if isinstance(x, list) else '')
        # df2

        # df3 = pd.json_normalize(df2)
        df2 = df2.iloc[::-1].reset_index(drop=True)
        df2.drop(columns=['comments'], inplace=True)
    else:
        df2 = pd.DataFrame()

    return df2
    # df2.to_csv('./temp.csv', index=False)

In [109]:
def main_func(matchid):
    comms_df = pd.DataFrame()
    for i in range(1, 3):
        CRICKET_AUS_COMMS = f"https://apiv2.cricket.com.au/web/views/comments?fixtureId={matchid}&inningNumber={i}&commentType=&overLimit=499&jsconfig=eccn%3Atrue&format=json"
        response = requests.get(CRICKET_AUS_COMMS, timeout=100)
        data = response.json()
        df = preprocess(data)
        comms_df = pd.concat([comms_df, df], ignore_index=True)

    comms_df.to_csv(f"./{matchid}.csv", index=False)
    # with open("./temp.json", "w") as f:
    #     json.dump(comms_data, f, indent=4)
    # comms_data.extend(data['inning'])
    # df = pd.read_json(data)

In [110]:
main_func(11291)

In [111]:
# import requests
# from datetime import datetime, timedelta

# # Base URL without matchId
# base_url = "https://polls.iplt20.com/?entity_matchId=74795&matchId="

# # Define your start and end times
# start_date = datetime(2024, 1, 1, 12, 0)  # Example start time
# end_date = datetime(2024, 1, 1, 12, 30)  # Example end time (half an hour later)

# # Generate timestamps for the range
# current_time = start_date
# time_increment = timedelta(seconds=1)  # Increment by 1 second


# while current_time <= end_date:
#     # Convert to Unix timestamp (seconds since epoch)
#     match_id = int(current_time.timestamp())
#     url = f"{base_url}{match_id}"

#     try:
#         # Send a GET request
#         response = requests.get(url, timeout=10)

#         # Log the response
#         print(f"URL: {url}")
#         print(f"Status Code: {response.status_code}")

#         if response.status_code == 200:
#             print(f"Response: {response.text}\n")
#         else:
#             print("Non-200 response received.\n")

#     except requests.exceptions.RequestException as e:
#         print(f"Request failed for URL {url}: {e}\n")

#     # Increment time
#     current_time += time_increment
